# Dot products, angles, and orthogonality

In this section, we review some of the basics of geometry in $\mathbb{R}^n$.

## Dot products and angles

### Review of dot products in $\mathbb{R}^n$

One of the most important quantities we compute in linear algebra are _inner products_, also called the _dot product_. 
For two vectors $x,y \in \mathbb{R}^n$, the inner product is the number


$$
x^\top y = \sum_{i=1}^n x_iy_i .
$$


Importantly, an inner product can be thought of as a linear function from $\mathbb{R}^n$ to $\mathbb{R}$: if we fix $y\in \mathbb{R}^n$, then the function $T_y(x) = x^\top y$ is clearly linear, since for any $x,x' \in \mathbb{R}^n$ and $\alpha \in \mathbb{R}$, we have


$$
T_y(x + \alpha x') = (x+\alpha x')^\top y = \sum_{i=1}^n (x_i + \alpha x'_i)y_i = \sum_{i=1}^n x_iy_i + \alpha\sum_{i=1}^nx'_iy_i = x^\top y + \alpha (x')^\top y = T_y(x) + \alpha T_y(x') .
$$


Dot products are also the basis of matrix multiplication: if $A \in \mathbb{R}^{n\times m}$ and $B\in \mathbb{R}^{m\times p}$ are matrices, and $a_1,\dots, a_n$ are the rows of $A$ and $b_1,\dots, b_p$ are the columns of $B$, then the $(i,j)$th element of $AB$ is just $a_i^\top b_j$. 

In this chapter, however, we will be interested in a more geometric interpretation of dot products, namely that they are used to compute the angle between two vectors.

### Computing angles between vectors with dot products

One of the most important facts about dot products is that they give us a way to compute the _angle_ $\theta$ between any two vectors $x,y\in \mathbb{R}^n$. 
This is due to the following important identity:


$$
x^\top y = \|x\|_2\|y\|_2\cos(\theta)
$$


Therefore we have that the angle $\theta$ can be found with 


$$
\theta = \arccos \left(\frac{x^\top y}{\|x\|_2 \|y\|_2}\right)
$$


Let's see a few examples in Python. 
First, let's write a useful function which finds the angle between any two vectors $x$ and $y$.

In [ ]:
import numpy as np

def angle_between_vectors(x,y):
    xty = np.dot(x, y)
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    theta = np.arccos(xty/(norm_x*norm_y))
    return theta

In this function we first compute the dot product $x^\top y$, then the norms of $x$ and $y$. 
Finally, we use `np.arccos` to take the arccosine of $x^\top y$ divided by $\|x\|_2\|y\|_2$ to find the angle. 

Let's look at a simple example to make sure this works. 
For example, consider $y = (1,1)$ and $x= (1,0)$. 
The vector $y$ is on the $45^\circ$ line, and $x$ is on the $x$-axis, and so the angle between them should be $45^\circ$, or $\pi/4$. 
Let's first define and plot these vectors.

In [ ]:
import matplotlib.pyplot as plt

x = np.array([1,0])
y = np.array([1,1])

origin = np.zeros(2)
plt.quiver(*origin, *x, label='x', scale=1, units='xy', color='blue')
plt.quiver(*origin, *y, label='y', scale=1, units='xy', color='red')
plt.grid()

plt.xlim(-1.5,1.5)
plt.ylim(-1.5,1.5)
plt.gca().set_aspect('equal')
plt.legend()
plt.show()

As expected, the two vectors appear to be at a $45^\circ$ angle. 
Let's use our function to check that this is correct.

In [ ]:
theta = angle_between_vectors(x,y)
print('angle between x and y: ', theta)
print('pi/4: ', np.pi/4)

As expected, the answer is $\pi/4$. 
Let's see what happens when we rotate the vectors by some angle. 
Recall that the matrix which rotates vectors by $\theta$ degrees is given by


$$
R_\theta = \begin{pmatrix}\cos(\theta) & -\sin(\theta)\\ \sin(\theta) & \cos(\theta)\end{pmatrix}
$$


Let's write a quick Python function to create this matrix.

In [ ]:
def rotation(theta):
    Rtheta = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
    return Rtheta

Now let's try rotating $y$ by $90^\circ$ (or $\pi/2$ radians) and $x$ by $45^\circ$ (or $\pi/4$) radians, and plot the newly rotated vectors.

In [ ]:
R90 = rotation(np.pi/2)
R45 = rotation(np.pi/4)

y_rotated = np.dot(R90, y)
x_rotated = np.dot(R45, x)

plt.quiver(*origin, *x_rotated, label='x rotated by pi/4', scale=1, units='xy', color='blue')
plt.quiver(*origin, *y_rotated, label='y rotated by pi/2', scale=1, units='xy', color='red')
plt.grid()

plt.xlim(-1.5,1.5)
plt.ylim(-1.5,1.5)
plt.gca().set_aspect('equal')
plt.legend()
plt.show()

The vectors appear to now by at a $90^\circ$ angle (which is what we would expect), but we can also verify this using our `angle_between_vectors` function.

In [ ]:
print('angle between x (rotated by pi/4) and y (rotated by pi/2): ', angle_between_vectors(x_rotated, y_rotated))

The answer is $\approx 1.5708$, which is just $\pi/2$. 

Vectors which are at a $90^\circ$ angle are special -- they are called _orthogonal_ (or _perpendicular_). 

### A gotcha

One point which is important to keep in mind when computing angles numerically is that we need to be careful when applying the $\arccos$ function in practice. To meaningfully interpret the angle $\arccos \left(\frac{x^\top y}{\|x\|_2 \|y\|_2}\right)$, we need to have that $-1\leq \frac{x^\top y}{\|x\|_2 \|y\|_2} \leq 1$. This is always true mathematically; however, numerically we could run into situations where $x$ and $y$ are parallel, but when we compute $\frac{x^\top y}{\|x\|_2 \|y\|_2}$ we obtain a number like $1.0000000000000002$. In this case, when we take the arccosine, we won't get a meaningful answer (since it is only defined for values in $[-1,1]$. Let's see a simple example.

In [ ]:
np.random.seed(123) #set seed for reproducibility

a = np.random.randn(10)
b = 1.2340124*a

angle_between_vectors(a,b)

 Here our function returns `nan`. We can check that the issue is numerical by computing $\frac{x^\top y}{\|x\|_2 \|y\|_2}$ directly.

In [ ]:
np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

Indeed, we get a number which is very slightly larger than $1$. In cases like this, it might be reasonable to try and round the value $\frac{x^\top y}{\|x\|_2 \|y\|_2}$.  


## Orthogonal vectors, bases, and matrices

Let's see what it means for two vectors to be orthogonal in terms of their dot product. 
If the angle between $x$ and $y$ is $90^\circ$ (or $\pi/2$ radians), then


$$
\arccos \left(\frac{x^\top y}{\|x\|_2 \|y\|_2}\right) = \pi/2 \iff \frac{x^\top y}{\|x\|_2 \|y\|_2} = \cos(\pi/2) = 0 \iff x^\top y= 0
$$


Therefore, two vectors $x$ and $y$ are _orthogonal_ if and only if their dot product is zero. 
Orthogonal vectors play an important role in linear algebra. 
In particular, we can define orthogonality between two vectors, between a vector and a subspace, and between two subspaces.

### Orthogonality between two subspaces

The notion of orthogonality is also easily extended to the span of two sets of vectors. For example, consider matrices $A \in \mathbb{R}^{n\times k}$ and $B \in \mathbb{R}^{n\times p}$, and let $a_1,\dots,a_k \in \mathbb{R}^n$ and $b_1,\dots, b_p\in \mathbb{R}^n$ be their columns. Given these matrices, we can define the following special subspaces of $\mathbb{R}^n$: $V_A = \text{span}(a_1,\dots,a_k)$ and $V_B = \text{span}(b_1,\dots,b_p)$. $V_A$ and $V_B$ are called the _column spaces_ of $A$ and $B$, respectively. Then we say that the subspaces $V_A$ and $V_B$ are _orthogonal_ if and only if $a_i^\top b_j = 0$ for all $i=1,\dots,k$ and $j=1,\dots,p$. Since we know that the the $(i,j)$th element of $A^\top B$ is exactly $a_i^\top b_j$, this is equivalent to saying that $A^\top B = \mathbf{0}$ where $\mathbf{0}$ is the $k\times p$ matrix of all zeros.

As a special case of this, we have that a vector $x = \begin{pmatrix}x_1\\ \vdots \\ x_n\end{pmatrix}$ is orthogonal to the column space of $A$ if $Ax=0$. In this case, we say that $x$ is in the _null space_ of $A$.  We also may use the notation $\text{null}(A)$ to denote the set of vectors $x$ such that $Ax= 0$. This set is also a subspace of $\mathbb{R}^n$. 

To understand this visually, let's consider the matrix $A$ given below


$$
A = \begin{pmatrix}1 & 0\\ 2& 3\\ 3&2\end{pmatrix}
$$


Let's define the columns as vectors $a_1 = \begin{pmatrix}1\\2\\3\end{pmatrix}$ and $a_2 = \begin{pmatrix}0\\3\\2\end{pmatrix}$.

In [ ]:
a1 = np.array([1,2,3])
a2 = np.array([0,3,2])

The subspace $V_A = \text{span}(a_1,a_2)$ is a plane in $\mathbb{R}^3$, which we can visualize with the code below.

In [ ]:
cp = np.cross(a1, a2)
a, b, c = cp

origin = np.zeros(3) 

d = np.dot(cp, origin)

fig = plt.figure(figsize=(16,7))
ax = fig.gca(projection='3d')

ax.quiver(*origin,*a1, length=1, arrow_length_ratio=0.1, colors='b', label='a1') 
ax.quiver(*origin,*a2, length=1, arrow_length_ratio=0.1, colors='g', label='a2') 

ax.legend()
xx, yy = np.meshgrid(np.arange(-5,8), np.arange(-5,8))
q = (d - a * xx - b * yy) / c

ax.plot_surface(xx, yy, q, alpha=0.5)
ax.view_init(15, 120)

plt.grid()
ax.set_xlim(-5,6)
ax.set_ylim(-5,6)
ax.set_zlim(-5,6)

plt.show()

Now let's find a vector that's orthogonal to the _subspace_ $V_A$. Such a vector $x = \begin{pmatrix}x_1 \\ x_2 \\ x_3\end{pmatrix}$ satisfies


$$
0 = a_1^\top x = x_1 + 2x_2 + 3x_3\;\;\; \text{ and }\;\;\; 0 = a_2^\top x = 3x_2 + 2x_3
$$


With some algebra, it is easy to verify that any vector $x$ of the form $x = \begin{pmatrix}\frac{5}{2}\alpha \\ \alpha \\ -\frac{3}{2}\alpha\end{pmatrix}$ will satisfy these conditions. Let's see an example with $\alpha = -2$.

In [ ]:
alpha = -2

x = np.array([5/2*alpha, alpha, -3/2*alpha])

Now let's plot $x$ and see that it is in fact orthogonal to the plane $V_A$.

In [ ]:
cp = np.cross(a1, a2)
a, b, c = cp

origin = np.zeros(3)

d = np.dot(cp, origin)

fig = plt.figure(figsize=(16,7))
ax = fig.gca(projection='3d')

ax.quiver(*origin,*a1, length=1, arrow_length_ratio=0.1, colors='b', label='a1') 
ax.quiver(*origin,*a2, length=1, arrow_length_ratio=0.1, colors='g', label='a2') 
ax.quiver(*origin,*x, length=1, arrow_length_ratio=0.1, colors='r', label='x') 

ax.legend()
xx, yy = np.meshgrid(np.arange(-5,8), np.arange(-5,8))
q = (d - a * xx - b * yy) / c

ax.plot_surface(xx, yy, q, alpha=0.5)
ax.view_init(15, 120)

plt.grid()
ax.set_xlim(-5,6)
ax.set_ylim(-5,6)
ax.set_zlim(-5,6)

plt.show()

As we can see, not only is the vector $x$ orthogonal to both the vectors $a_1$ and $a_2$, but it is in fact orthogonal to the entire subspace $V_A$. 


### Orthogonal and orthonormal sets and bases

Before defining what an orthogonal basis is, we first cover what it means for a set of vectors to be orthogonal. 
A set $V = \{v_1,\dots,v_k\}$ of vectors $v_1,\dots,v_k \in \mathbb{R}^n$ is called an _orthogonal set_ if $v_i^\top v_j = 0$ for any $i\neq j$. 
Since a basis is just a linearly indepedent set of vectors which spans a vector space, an _orthogonal basis_ is simply a basis which is also orthogonal. 

For example, the standard basis in $\mathbb{R}^n$ is orthogonal. 
Here the standard basis is the set $E = \{e_1,\dots, e_n\}$ where 


$$
e_i = \begin{pmatrix}0\\ \vdots \\1\\ \vdots \\ 0\end{pmatrix}
$$


i.e., it is $1$ in the $i^{th}$ coordinate and $0$ elsewhere. 
Then since for $i\neq j$, the vectors $e_i$ and $e_j$ don't have any non-zero entries in common, it is easy to see that $e_i^\top e_j = 0$. 
Hence the set $E$ is an orthogonal set, and since we know it is a basis, it is also an orthogonal basis!

It turns out that requiring a set to be orthogonal an linearly independent is superfluous: every orthogonal set is automatically linearly independent (though the converse is not true). 
We won't prove this here, but it is a good exercise to try on your own. 
Therefore, an orthogonal basis for $\mathbb{R}^n$ is any orthogonal set of vectors which spans $\mathbb{R}^n$. 
While the standard basis is an easy example, it is not the only example. 
For instance, any rotation of the standard basis is also an orthogonal basis. 

For example, let's start with the standard basis in $\mathbb{R}^2$:


$$
e_1 = \begin{pmatrix}1\\ 0\end{pmatrix},\;\;\; e_2 = \begin{pmatrix}0\\ 1\end{pmatrix}
$$


Let's define these in Python.

In [ ]:
e1 = np.array([1,0])
e2 = np.array([0,1])

Now, let's try rotating these vectors by $45^\circ$. 
To do this, we can use the rotation matrix that we defined in the previous section.

In [ ]:
R45 = rotation(np.pi/4)

v1 = np.dot(R45, e1)
v2 = np.dot(R45, e2)

print('v1 = ', v1)
print('v2 = ', v2)

This gives the vectors


$$
v_1 = \begin{pmatrix}1/\sqrt{2}\\ 1/\sqrt{2}\end{pmatrix},\;\;\; v_2 = \begin{pmatrix}-1/\sqrt{2}\\ 1/\sqrt{2}\end{pmatrix}
$$


Let's check that these vectors are still orthogonal, but in two different ways. 
First, we can use the `angle_between_vectors` function to check that the angle between $v_1$ and $v_2$ is indeed $90^\circ$.

In [ ]:
angle_between_vectors(v1,v2)

As expected, we get $\pi/2$ back. 
On the other hand, we can also check that $v_1^\top v_2 = 0$:

In [ ]:
np.dot(v1,v2)

The set $\{v_1,v_2\}$ is not only an orthogonal set, but it is in fact an _orthonormal set_. 
This means that the vectors are orthogonal, but are also unit vectors. 
Given an orthogonal set, it is easy to construct an othonormal set by simply dividing each vector in the set by its norm. 
If $V = \{v_1,\dots, v_k\}$ is an orthonormal set, then we have


$$
v_i^\top v_j = \begin{cases}1 & \text{if } i=j\\ 0 & \text{if } i\neq j\end{cases}
$$


### Orthogonal matrices

Suppose we have an orthonormal set of vectors $V = \{v_1,\dots,v_k\}$, where $v_i \in \mathbb{R}^n$. 
We can stack these vectors into a $n\times k$ matrix $Q$, with $v_i$ being the $i$th column of $Q$:


$$
Q = \begin{pmatrix}|&|&&|\\ v_1 & v_2 & \cdots & v_k\\ |&|&&|\end{pmatrix}
$$


Let's see what happens what we compute the matrix $Q^\top Q$. 
Since $Q$ is $n\times k$, $Q^\top Q$ is a $k\times k$. 
As we mentioned earlier in this section, the $(i,j)$th entry of $Q^\top Q$ is just $v_i^\top v_j$. 
Therefore, we have the following description of the entries of $Q^\top Q$:


$$
[Q^\top Q]_{ij} = v_i^\top v_j = \begin{cases}1 & \text{if } i=j\\ 0 & \text{if } i\neq j\end{cases}
$$


Therefore, the diagonal entries of $Q$ are $1$ and the off-diagonal entries are 0. 
Said another way, $Q^\top Q = I$ -- i.e., it is the identity matrix on $\mathbb{R}^k$. 
Any matrix $Q$ which satisfies $Q^\top Q = I$ is called an _orthogonal matrix_. 
Equivalently, an orthogonal matrix is a matrix whose columns form an orthonormal set.

**Remark:** One point that often causes confusion is that an orthogonal matrix $Q$ doesn't just have _orthogonal_ columns, but rather _orthonormal_ columns. 
Perhaps it would make more sense to call such a matrix $Q$ an orthonormal matrix, but unfortunately this terminology is now standard, and so it is important to remember this distinction.

Let's see a few examples of orthogonal matrices in Python. 
We can start with our set $\{v_1,v_2\}$ that we defined above, from rotating the standard basis by $45^\circ$.

In [ ]:
Q = np.stack([v1,v2], axis=1)
Q

Let's check that $Q^\top Q = I$.

In [ ]:
np.dot(Q.T, Q).round(8)

Here we round to 8 decimals to avoid issues of numerical precision (though in practice you may want to be careful when doing this). 
In fact, since the matrix $Q$ is square in this case, we observe that $Q^{-1} = Q^\top$. 
This relationship is true for any square orthogonal matrix.

We can also look at a bigger example. 
For now, we won't explain how the below method works; we will cover this in the next section of the workbook.

In [ ]:
n = 10
k = 5

A = np.random.randn(n, k)
Q, _ = np.linalg.qr(A)
Q

Let's check that the $10\times 5$ matrix $Q$ is in fact orthogonal, by checking that $Q^\top Q = I$.

In [ ]:
np.dot(Q.T, Q).round(8)

Indeed it is.

In the above code, we used something called a $QR$ decomposition to generate the othogonal matrix $Q$. 
This is a generic method which we can use to find an orthogonal set from any given set of vectors in $\mathbb{R}^n$. 
This is one of the most important decompositions in linear algebra, and we will discuss it in much more detail in the next sections of the workbook.